In [22]:
import pandas as pd
from os.path import exists

# read files from directory
file_list = []

for file in sorted(os.listdir('static/img/portfolio')):
    file_list.append(file)

df_dir = pd.DataFrame(file_list, columns=['Files'])
df_old = pd.read_csv('processed.csv')
df_new = df_old.merge(df_dir, how = 'outer' ,indicator=True).loc[lambda x : x['_merge']=='left_only']

print(df_new)


Empty DataFrame
Columns: [Files, _merge]
Index: []
